### Creating variables for the api urls and logging in using personal details

In [80]:
#############
## imports ##
#############

import pandas as pd
from datetime import datetime as dt
import requests
import numpy as np

######################################################################################

###########################
## URLs an Login Details ##
###########################

my_team_id = '3366741'

## URLs

## login url to create a session using my login details ## 
login_url = 'https://users.premierleague.com/accounts/login/'

## api url for the main dataset on player stats 
main_url = 'https://fantasy.premierleague.com/api/bootstrap-static/'

## api for my team
team_url = 'https://fantasy.premierleague.com/api/my-team/3366741/'

## api for ???
entry_url = 'https://fantasy.premierleague.com/api/entry/{team-id}/'

## api for fixtures
fix_url = 'https://fantasy.premierleague.com/api/fixtures/'

session = requests.session()

## login details ##
payload = {
 'password': 'Football!159',
 'login': 'andrew_c_morris@hotmail.co.uk',
 'redirect_uri': 'https://fantasy.premierleague.com/a/login',
 'app': 'plfpl-web'
}
session.post(login_url, data=payload)


<Response [200]>

### Creating data frames and csv outputs required for the excel  based model

In [57]:
## CLUB DATAFRAME AND CSV EXTRACT ##
### Creating a dataframe for the club data and saving as csv ###

############################
#### dataframe creation ####
############################

## Converting the data from the url to json then pulling out the teams data to a separate 'club' data frame
main_req = requests.get(main_url)
main_json = main_req.json()
main_json.keys()
club_df = pd.DataFrame(main_json['teams'])

#####################
##### csv output ####
#####################

## Specifying the filename
filename = 'FPL_Clubs-'

## Dateime to add to the filname
datetime = dt.now().strftime("%Y%m%d-%H%M%S")

## Location to export the CSV to
path = 'C:\\Python CSV Output\\'+filename+datetime+'.csv'

## Saved dataframe as csv
club_df.to_csv(path,
          index = False)

In [90]:
### PLAYER DATA EXTRACT ###
### Creating a dataframe for the player data and saving as csv ###

############################
#### Dataframe Creation ####
############################

full_player_df = pd.DataFrame(main_json['elements']) # Player data
position_types_df = pd.DataFrame(main_json['element_types'])
team_df = pd.DataFrame(main_json['teams'])
events_df = pd.DataFrame(main_json['events'])

print(team_df.head())

# Updating some of the fields: 
    # Pulling through position and team name instead of 'id', and changing data types
    # Converting data types of 'value_season' ans 'selected_by_percent'
full_player_df['position'] = full_player_df.element_type.map(position_types_df.set_index('id').singular_name)
full_player_df['position_abv'] = full_player_df.element_type.map(position_types_df.set_index('id').singular_name_short)
full_player_df['team'] = full_player_df.team.map(team_df.set_index('id').name)
full_player_df['team_abv'] = full_player_df.team.map(team_df.set_index('id').short_name)
full_player_df['value_season'] = full_player_df.value_season.astype(float)
full_player_df['selected_by_percent'] = full_player_df.selected_by_percent.astype(float)

# Creating a slimline version of the elements table with relevant data points required for players
slim_player_df = full_player_df[['first_name',
                              'second_name',
                              'position',
                              'position_abv',
                              'team',
                              'team_abv',
                              'selected_by_percent',
                              'status',
                              'now_cost',
                              'minutes',
                              'transfers_in',
                              'total_points',
                              'value_season',
                             ]]

print(slim_player_df.head())

######## Show Datatypes Of Dataframe ##########
# dataTypeSeries = slim_player_df.dtypes
# print('Data type of each column of Dataframe :')
# print(dataTypeSeries)
###############################################

#####################
##### csv output ####
#####################

## Specifying the filename and datetime and path (Location to export the CSV to)
player_filename = 'FPL_Players-'
player_datetime = dt.now().strftime("%Y%m%d-%H%M%S")
player_path = 'C:\\Python CSV Output\\'+player_filename+player_datetime+'.csv'

# ## Exporting dataframe as csv
# slim_player_df.to_csv(player_path, index = False, encoding="utf8")

   code  draw  form  id  loss         name  played  points  position  \
0     3     0  None   1     0      Arsenal       0       0         0   
1     7     0  None   2     0  Aston Villa       0       0         0   
2    36     0  None   3     0     Brighton       0       0         0   
3    90     0  None   4     0      Burnley       0       0         0   
4     8     0  None   5     0      Chelsea       0       0         0   

  short_name  ...  team_division unavailable  win  strength_overall_home  \
0        ARS  ...           None       False    0                   1190   
1        AVL  ...           None       False    0                   1050   
2        BHA  ...           None       False    0                   1070   
3        BUR  ...           None       False    0                   1140   
4        CHE  ...           None       False    0                   1200   

   strength_overall_away  strength_attack_home  strength_attack_away  \
0                   1270              

In [ ]:
### FIXTURE DATA EXTRACT ###
### Creating a dataframe for the fixtures data and saving as csv ###

############################
#### dataframe creation ####
############################

## Getting fixture data from the api url
fix_req = requests.get(fix_url)

## Converting that data to json then dataframe
fix_json = fix_req.json()
fix_df = pd.DataFrame(fix_json)

## Slimming down the dataframe to only the columns we need
slim_fix_df = fix_df[['event','kickoff_time','team_a','team_h','team_h_difficulty','team_a_difficulty']]

# Renaming a few columns
slim_fix_df = slim_fix_df.rename(columns={"event": "Gameweek", "team_a":"Away_Team","team_h":"Home_Team" })


#####################
##### csv output ####
#####################

## Specifying the filename and datetime
filename3 = 'FPL_FIXTURES-'
datetime = dt.now().strftime("%Y%m%d-%H%M%S")

## Location to export the CSV to
path = 'C:\\Python CSV Output\\'+filename3+datetime+'.csv'

## Exporting dataframe as csv
fix_df.to_csv(path,
          index = False)

In [ ]:
## MY TEAM PICKS DATAFRAME AND CSV EXTRACT ##
### Creating a dataframe for my team picks data and saving as csv ###

############################
#### dataframe creation ####
############################

## Getting my team data from the api url, using session as login details required
my_team_ses = (session.get('https://fantasy.premierleague.com/api/my-team/3366741'))

## Converting that data to json then dataframe
my_team_json = my_team_ses.json()
my_team_picks_df = pd.DataFrame(my_team_json['picks'])


#####################
##### csv output ####
#####################

## Specifying the filename and datetime
filename4 = 'FPL_My_Team_Picks-'
datetime = dt.now().strftime("%Y%m%d-%H%M%S")

## Location to export the CSV to
path = 'C:\\Python CSV Output\\'+filename+datetime+'.csv'

## Exporting dataframe as csv
club_df.to_csv(path,
          index = False)

### Random Bits

In [ ]:
phases_df = pd.DataFrame(main_json['phases'])
element_stats_df = pd.DataFrame(main_json['element_stats'])

In [ ]:
## create a dataframe for the events (gameweek summary) and save to csv ##

events_df = pd.DataFrame(main_json['events'])

## specifying the filename
filename3 = 'FPL_EVENTS-'

## Dateime to add to the filname
datetime = dt.now().strftime("%Y%m%d-%H%M%S")

## Location to export the CSV to
path = 'C:\\Python CSV Output\\'+filename3+datetime+'.csv'

## Saved dataframe as csv
events_df.to_csv(path,
          index = False)